In [1]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']


In [2]:
tss_annotation = doc['tss_annotation']

In [3]:
from collections import defaultdict
import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 15)


In [4]:
meta_samples = pd.read_csv('../raw_metaSamples.tsv',sep="\t",index_col=0)
meta_samples.head()

,Tissue,Experiment,Name,Full
CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001_R1_001.fastq.gz,BMDM1hKLA,ATAC,BMDM1hKLA_ATAC1,CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001...
CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R1_001.fastq.gz,BMDM1hKLA,ATAC,BMDM1hKLA_ATAC2,CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R...
CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_001.fastq.gz,BMDM1hKLA,GRO,BMDM1hKLA_GRO1,CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_...
CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_001.fastq.gz,BMDM1hKLA,GROCap,BMDM1hKLA_GROCap1,CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_...
CHBMDMwt_GRO_JHS1026_SD_ATTCCT_S41_L003_R1_001.fastq.gz,BMDMwt,GRO,BMDMwt_GRO1,CHBMDMwt_GRO_JHS1026_SD_ATTCCT_S41_L003_R1_001...


## Load anno_peaks. 
Later will filter based on Distance to TSS

## Load peaks_tissues matrix
Later will be used to merge two data types in a tissue and for more confident peaks if they have more than 1 tissues present

## Load tss

## Load txn_df

## Load geneID_df



In [5]:
anno_peaks = pd.read_csv('Results/merged/peaks_with_tss_distances.tsv',sep='\t',index_col=0)
anno_peaks.head()

,Chr,Start,End,Strand,Stat,Annotation,Nearest TSS,Distance to TSS,isSameStrand,Nearest gene,Nearest gene_id
ID,,,,,,,,,,,
0,picr_9,30423275,30423564,+,10.506897,"exon (asmbl_83977, exon 1 of 1)",asmbl_83977,4098.0,False,SALL1,gene_29627
1,picr_27,14937806,14938107,-,22.268621,promoter-TSS (asmbl_40109),asmbl_40105,-159.0,False,RBM25,gene_13991_3
2,picr_2,5127066,5127335,+,1.732333,"intron (asmbl_29716, intron 9 of 9)",asmbl_29719,-8640.0,True,FAM83C,gene_10160
3,picr_15,26110487,26110670,+,1.659667,"intron (asmbl_22566, intron 1 of 4)",asmbl_22566,24071.0,True,ZNRF1,gene_7657_3
4,picr_0,73670402,73670628,+,1.694250,Intergenic,asmbl_3271,-74096.0,True,MAN2A1,gene_975


In [6]:
peaks_tissue = pd.read_csv("Results/merged/samples.merge.peaksexpression",index_col=0,sep="\t")
peaks_tissue.head()

,f04_peaks/Brain_START2_and_Brain_STARTinput1,f04_peaks/Brain_GROCap1_and_Brain_GRO1,f04_peaks/FemaleReproductive_START1_and_FemaleReproductive_STARTinput1,f04_peaks/Kidney_START1_and_Kidney_STARTinput1,f04_peaks/Muscle_START2_and_Muscle_STARTinput1,f04_peaks/Heart_START1_and_Heart_STARTinput1,f04_peaks/Lung_GROCap1_and_Lung_GRO1,f04_peaks/Liver_START2_and_Liver_STARTinput1,f04_peaks/MiscOrgans_START2_and_MiscOrgans_STARTinput1,f04_peaks/Muscle_START1_and_Muscle_STARTinput1,...,f04_peaks/Brain_START1_and_Brain_STARTinput2,f04_peaks/FemaleReproductive_START2_and_FemaleReproductive_STARTinput1,f04_peaks/Pancreas_START2_and_Pancreas_STARTinput1,f04_peaks/Brain_START3_and_Brain_STARTinput1,f04_peaks/Kidney_GROCap1_and_Kidney_GRO1,f04_peaks/Brain_START1_and_Brain_STARTinput1,f04_peaks/CHO_GROCap1_and_CHO_GRO1,f04_peaks/BMDMwt_GROCap1_and_BMDMwt_GRO1,f04_peaks/Heart_START2_and_Heart_STARTinput1,f04_peaks/Lung_START1_and_Lung_STARTinput1
0,15.5,5.12,0.0,26.50,22.50,2.34,0.0,0.0,9.37,1.93,...,23.00,16.10,10.10,11.60,1.05,23.00,3.12,4.7,0.0,14.4
1,9.6,50.80,0.0,4.09,7.78,3.68,0.0,0.0,6.30,5.78,...,5.25,5.85,4.77,2.48,103.00,5.25,46.00,65.6,0.0,10.7
2,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,1.15,0.00,0.00,0.0,0.0,0.0
3,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,1.94,0.00,0.00,0.0,0.0,0.0
4,0.0,0.00,0.0,0.00,0.00,0.00,0.0,2.5,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.79,0.0,0.0,0.0


#### Read in tss and txn_df

In [7]:
tss = pd.read_csv(tss_annotation,sep='\t',index_col=0)

tss.head()

,Chr,Start,End,Strand,ID,Parent,gene_id,gene,transcript_id,cs,Length,actual_start
ID=asmbl_2;Parent=gene_1_2;gene_id=gene_1_2;gene=SERHL;transcript_id=asmbl_2;cs=2,picr_0,176,38694,+,asmbl_2,gene_1_2,gene_1_2,SERHL,asmbl_2,2,38518,176
ID=asmbl_4;Parent=gene_2;gene_id=gene_2;gene=RRP7A;transcript_id=asmbl_4;cs=1,picr_0,7715,46630,-,asmbl_4,gene_2,gene_2,RRP7A,asmbl_4,1,38915,46630
ID=asmbl_7;Parent=gene_2;gene_id=gene_2;gene=RRP7A;transcript_id=asmbl_7;cs=1,picr_0,7759,44597,-,asmbl_7,gene_2,gene_2,RRP7A,asmbl_7,1,36838,44597
ID=asmbl_8;Parent=gene_2;gene_id=gene_2;gene=RRP7A;transcript_id=asmbl_8;cs=1,picr_0,7767,44597,-,asmbl_8,gene_2,gene_2,RRP7A,asmbl_8,1,36830,44597
ID=asmbl_9;Parent=gene_2;gene_id=gene_2;gene=RRP7A;transcript_id=asmbl_9;cs=1,picr_0,7789,44568,-,asmbl_9,gene_2,gene_2,RRP7A,asmbl_9,1,36779,44568


In [8]:
txn_df = pickle.load(open('Results/tss_annotation/txn_df_02.p','rb'))# pd.read_csv('Results/txn_df.csv',index_col=0)
txn_df.head()

,peaks,closest_to_X_numberOfPeaks,bin_-1000_1000,sameStrand_bin_-1000_1000,samples,minDistance,minDistancePeak,maxValue,maxPeakId,maxSample,Chr,Start,End,Strand,gene_id,gene,transcript,maxSamples,hasGene,Number of SS
asmbl_2,[],0,[],[],[],inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],False,0
asmbl_4,"[215685, 249167, 398989, 483229]",4,[215685],[215685],[CHO_START1_and_CHO_STARTinput1],706.000000,215685,3.04,215685,NaN,picr_0,45849,45999,-,gene_2,RRP7A,asmbl_4,[CHO_START1_and_CHO_STARTinput1],True,1
asmbl_7,[104402],1,[104402],[],[],inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],False,0
asmbl_8,[],0,[],[],[],inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],False,0
asmbl_9,[],0,[],[],[],inf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],False,0


In [9]:
geneID_df = pickle.load(open('Results/tss_annotation/geneID_df_02.p','rb'))

### Merge tissue columns

In [10]:
def merge_two_columns(df, mapping_dict):
    '''Function that merges columns by taking the mean of them.
    Merges based on if they have the same element in the meta_samples meta_column.
    Returns:
        new_df: Dataframe but with the columns of interest merged. Also column names are now based
                on the unique meta_samples[meta_column].
    '''

    vals = mapping_dict.keys()#np.unique(meta_samples[meta_column].values)
    new_df = pd.DataFrame(index=df.index, columns=vals)
    for i in vals:
        if not mapping_dict[i] == []:#(meta_samples[meta_column] == i).any():
            new_col = (df.loc[:, mapping_dict[i]])
            new_col = new_col.mean(axis=1)
            new_df.loc[:, i] = new_col
    new_df = new_df.loc[:, ~(new_df.isnull().all())]
    return new_df

In [11]:
tissue_to_peaks = defaultdict(list)
for val in peaks_tissue.columns.values:
    count = 0
    for t in np.unique(meta_samples["Tissue"].values):
        if t in val:
            count += 1
            tissue_to_peaks[t].append(val)
            if count > 1:
                print("This sample has multiple tissues associated with it")
                print("Sample: ",val)

In [12]:
peaks_tissue_merged = merge_two_columns(peaks_tissue,mapping_dict=tissue_to_peaks)

In [13]:
peaks_tissue_merged.head()

,Heart,BMDMwt,Spleen,FemaleReproductive,CHO,Brain,Lung,Liver,MiscOrgans,Muscle,Kidney,Pancreas
0,1.17,4.7,3.455,8.050,4.1820,15.045714,4.800000,14.380000,10.485,12.215,10.576667,6.875
1,1.84,65.6,5.195,2.925,48.2000,12.208571,3.566667,25.080000,7.100,6.780,37.786667,3.375
2,0.00,0.0,0.000,0.000,0.1494,0.000000,0.000000,1.100000,0.000,0.000,0.383333,0.000
3,0.00,0.0,0.000,0.000,0.0798,0.000000,0.000000,0.880000,0.000,0.000,0.646667,0.000
4,0.00,0.0,0.000,0.000,0.3274,0.000000,0.000000,1.713333,0.000,0.000,0.000000,0.000


### Create directory of different peak groups

In [24]:
if not os.path.exists("Results/peak_groups"):
    os.mkdir("Results/peak_groups")

In [15]:
def save_peak_as_bed(peaks_df,f_save, name='index',score_col = 'Peak Score',is_sort = True):
    ## Assumes there is Chr,Start,End,Name is index
    ## Peak file is 1-index, bed files are 0, so need to shift everything by one
    
    if name == 'index':
        peaks_df.loc[:,'Name'] = peaks_df.index
    else:
        peaks_df.loc[:,'Name'] = peaks_df[name]
    
    if score_col == '': #If no scoring col, set to 0
        peaks_df.loc[:,'Score'] = 0
        score_col = 'Score'
        
    peaks_df.loc[:,'Start'] = peaks_df['Start'] - 1
    peaks_df.loc[:,'End'] = peaks_df['End'] - 1
    
    peaks_df = peaks_df[['Chr','Start','End','Name',score_col,'Strand']]
    
    if is_sort:
        peaks_df.sort_values(['Chr','Start'],inplace=True)
        
    if not f_save.endswith('.bed'):
        f_save = f_save + '.bed'
    peaks_df.to_csv(f_save ,sep='\t',index=False,header=None)
    return
    

## a) Separate confident promoters and confident ncRNA 
Define by distance to RefSeq being 0 and ncRNA far >40kb from a gene and in at least two tissues

### c) Predicted promoters vs RefSeq promoters

### d) Promoters organized based on distance to refseq (so heatmap is not sorted by heat but by distance)

### e) Only look at peaks specific for each tissue

In [16]:
def filter_only_with_divergent(df,divergent_threshold = 500 ):
    inds_to_keep = []
    for ind,val in df[df['isSameStrand']].iterrows():
        curr = df[(val['Chr'] == df['Chr']) & (df['isSameStrand'] == False)]
        if val['Strand'] == '-':
            curr = curr[(val['End'] - curr['Start'] < divergent_threshold) & 
                       (val['End'] - curr['Start'] > 0)]
        else: 
            curr = curr[(val['Start'] - curr['End'] < divergent_threshold) &
                       (val['Start'] - curr['End'] > 0 ) ]

        ## If there are divergent peaks, add this peak
        if len(curr) > 0: 
            inds_to_keep.append(ind)
    
    df = df.loc[inds_to_keep]
    return df



In [28]:
def tissue_specific_promoter(t,
                             anno_peaks,
                             peaks_tissue_merged,
                             txn_df,tss,
                             peak_count_thresh = 1, 
                             prom_bin=[-1000, 1000],
                             enhancer_d=40000,
                             divergent_thr=500,
                             outfolder='Results/peak_groups/'):

    ''' Function to create bed files relevant for a specific tissue
        Will save:
        a) All peaks that were found in tissue
        b) Peaks within prom_bin distance from reference & has divergent peak nearby (divergent_thr distance)
        c) Same as b, but no divergent restriction
        d) Same as c, but the peak center is now the annnotated start site instead of experimental
        e) ncRNA, possibly eRNA that are greater than enhancer_d from any annotated start sites.
        f) no_peaks: genes with no peaks nearby, even divergent peaks'''
    
    
    
    t_peaks = peaks_tissue_merged[peaks_tissue_merged.loc[:, t] > peak_count_thresh].index #Peaks in that tissue
    t_peaks = anno_peaks.loc[t_peaks]
    t_peaks = t_peaks[(~(t_peaks['Distance to TSS'] == np.infty))] ## Remove peaks that are on a scaffold with no gene

    ### Absence of peaks for that tissue.
    no_t_peaks = txn_df[txn_df['samples'].apply(lambda x: np.array(map(lambda y: t not in y,x)).any())]
    no_t_peaks = tss[tss['transcript_id'].apply(lambda x: x in no_t_peaks.index)]
    #no_t_peaks = peaks_tissue_merged[peaks_tissue_merged.loc[:, t] <= peak_count_thresh].index #Peaks not in the tissue
    #no_t_peaks = anno_peaks.loc[no_t_peaks]
    


    t_prom = t_peaks[(t_peaks['Distance to TSS'] > prom_bin[0])
                     & (t_peaks['Distance to TSS'] < prom_bin[1])]
    t_prom_isSame = t_prom[t_prom['isSameStrand']]
    
    
    t_enh = t_peaks[np.abs(t_peaks['Distance to TSS']) > enhancer_d]
    t_prom_div = filter_only_with_divergent(
        t_prom, divergent_threshold=divergent_thr)
    t_enh_div = filter_only_with_divergent(
        t_enh, divergent_threshold=divergent_thr)

    t_prom_div = t_prom_div.sort_values(['Distance to TSS'])
    
    #Look at RefSeq vs experimental TSS
    tss_withPeaks = t_prom_isSame['Nearest TSS'].unique()
    refSeq = tss[tss['transcript_id'].apply(lambda x: x in tss_withPeaks)] 
    # Dont want the whole gene length, just the start of the peak 
    refSeq['Start'] = refSeq['actual_start'] #actual_start takes care of +/- strand
    refSeq['End'] = refSeq['actual_start']
    
    ## Saving bed files
    out_dir = os.path.join(outfolder, t)
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    
    save_peak_as_bed(
        t_peaks,
        os.path.join(out_dir, 'all_peaks'),
        name='index',
        score_col='Stat')    
    
    save_peak_as_bed(
        t_prom_div,
        os.path.join(out_dir, 'confident_promoters'),
        name='index',
        score_col='Stat')
    
    save_peak_as_bed(
        t_prom_isSame,
        os.path.join(out_dir, 'confident_promoters_noDiv'),
        name='index',
        score_col='Stat')  
    
    save_peak_as_bed(
        refSeq,
        os.path.join(out_dir, 'confident_promoters_noDiv_PICR'),
        name='index',
        score_col='cs')
    
    save_peak_as_bed(
        t_enh_div,
        os.path.join(out_dir, 'confident_enhancers'),
        name='index',
        score_col='Stat')
    
    save_peak_as_bed(
        no_t_peaks,
        os.path.join(out_dir, 'no_peaks'),
        name='index',
        score_col='cs')
    
    
    return

In [18]:
tissues_with_atac = ['CHO','Brain','Liver','Lung','Kidney','Spleen','BMDMwt']#,'BMDM1hKLA']

In [29]:
%%time
for t in tissues_with_atac:
    print(t)
    tissue_specific_promoter(
        t=t,
        anno_peaks=anno_peaks,
        peaks_tissue_merged=peaks_tissue_merged,
        txn_df=txn_df,
        tss=tss)

CHO


/home/isshamie/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/isshamie/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/isshamie/software/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

Brain
Liver
Lung
Kidney
Spleen
BMDMwt
CPU times: user 14min 5s, sys: 4.23 s, total: 14min 9s
Wall time: 14min 5s


# TSS peaks near ATAC peaks. 
Using Bedtools

### Sort ATAC bed files

In [38]:
## Sort ATAC bed files
for t in tissues_with_atac:
    print(t)
    atac_f = glob.glob(os.path.join(data_folder, t, 'ATAC/out/peak/macs2/overlap/optimal_set/*narrowPeak.gz'))[0]
    atac_f_out = atac_f.replace('.gz','') + '.sort'
    atac_f_out = os.path.join("Results/ATAC_results/",t,os.path.basename(atac_f).replace('.gz','.sort'))
    if not os.path.exists(os.path.join("Results/ATAC_results/",t)):
        os.mkdir(os.path.join("Results/ATAC_results/",t))
    
    cmd = 'bedtools sort -i %s > %s' % (atac_f,atac_f_out)   
    print(cmd)
    os.system(cmd)

CHO
bedtools sort -i /data/isshamie/TSS/1015_Processed/CHO/ATAC/out/peak/macs2/overlap/optimal_set/ATAC_ppr.naive_overlap.narrowPeak.gz > Results/ATAC_results/CHO/ATAC_ppr.naive_overlap.narrowPeak.sort
Brain
bedtools sort -i /data/isshamie/TSS/1015_Processed/Brain/ATAC/out/peak/macs2/overlap/optimal_set/ATAC_rep1-pr.naive_overlap.narrowPeak.gz > Results/ATAC_results/Brain/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
Liver
bedtools sort -i /data/isshamie/TSS/1015_Processed/Liver/ATAC/out/peak/macs2/overlap/optimal_set/ATAC_rep1-pr.naive_overlap.narrowPeak.gz > Results/ATAC_results/Liver/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
Lung
bedtools sort -i /data/isshamie/TSS/1015_Processed/Lung/ATAC/out/peak/macs2/overlap/optimal_set/ATAC_rep1-pr.naive_overlap.narrowPeak.gz > Results/ATAC_results/Lung/ATAC_rep1-pr.naive_overlap.narrowPeak.sort
Kidney
bedtools sort -i /data/isshamie/TSS/1015_Processed/Kidney/ATAC/out/peak/macs2/overlap/optimal_set/ATAC_rep1-pr.naive_overlap.narrowPeak.gz > Resul

### Get peaks close to TSS within distance of 500 bp

In [32]:
def peaks_closest(f_a, f_b, out_f, dist_thresh):
    '''Function that calls bedtools closest to find the closest peak of two bed files, and awk to filter those within distance dist_thresh'''

    cmd = "bedtools closest -d -t first -a %s -b %s | awk '{if ($NF < %d) print }' > %s" % (
       f_a, f_b, dist_thresh,out_f) #t is to only report one of the atac peaks
    
#     cmd = "bedtools closest -d -t first -a %s -b %s  > %s" % (
#         f_a, f_b,out_f)
    
    print(cmd)
    os.system(cmd)
    return

In [40]:
# Intersect ATAC and TSS
dist_thresh = 500 #500 basepairs left or right
outfolder = 'Results/peak_groups/'
f_names = [
    'all_peaks','confident_promoters', 'confident_enhancers', 'confident_promoters_noDiv',
    'confident_promoters_noDiv_PICR', 'no_peaks'
]


for t in tissues_with_atac:
    print(t)
    ## Outdirectory to save in
    out_dir = os.path.join(outfolder, t)
    
    #ATAC peak file
    atac_out = glob.glob(os.path.join("Results/ATAC_results", t , '*sort'))[0]
    
    ## The different TSS files
    for f in f_names:
        f_in = os.path.join(out_dir, f) + '.bed'
        out_f = '%s_atac_overlap_%dbp.bed' % (f_in,dist_thresh)
        peaks_closest(f_in,atac_out,out_f,dist_thresh=dist_thresh)

CHO
bedtools closest -d -t first -a Results/peak_groups/CHO/all_peaks.bed -b Results/ATAC_results/CHO/ATAC_ppr.naive_overlap.narrowPeak.sort | awk '{if ($NF < 500) print }' > Results/peak_groups/CHO/all_peaks.bed_atac_overlap_500bp.bed
bedtools closest -d -t first -a Results/peak_groups/CHO/confident_promoters.bed -b Results/ATAC_results/CHO/ATAC_ppr.naive_overlap.narrowPeak.sort | awk '{if ($NF < 500) print }' > Results/peak_groups/CHO/confident_promoters.bed_atac_overlap_500bp.bed
bedtools closest -d -t first -a Results/peak_groups/CHO/confident_enhancers.bed -b Results/ATAC_results/CHO/ATAC_ppr.naive_overlap.narrowPeak.sort | awk '{if ($NF < 500) print }' > Results/peak_groups/CHO/confident_enhancers.bed_atac_overlap_500bp.bed
bedtools closest -d -t first -a Results/peak_groups/CHO/confident_promoters_noDiv.bed -b Results/ATAC_results/CHO/ATAC_ppr.naive_overlap.narrowPeak.sort | awk '{if ($NF < 500) print }' > Results/peak_groups/CHO/confident_promoters_noDiv.bed_atac_overlap_500bp.

bedtools closest -d -t first -a Results/peak_groups/Spleen/confident_enhancers.bed -b Results/ATAC_results/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort | awk '{if ($NF < 500) print }' > Results/peak_groups/Spleen/confident_enhancers.bed_atac_overlap_500bp.bed
bedtools closest -d -t first -a Results/peak_groups/Spleen/confident_promoters_noDiv.bed -b Results/ATAC_results/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort | awk '{if ($NF < 500) print }' > Results/peak_groups/Spleen/confident_promoters_noDiv.bed_atac_overlap_500bp.bed
bedtools closest -d -t first -a Results/peak_groups/Spleen/confident_promoters_noDiv_PICR.bed -b Results/ATAC_results/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort | awk '{if ($NF < 500) print }' > Results/peak_groups/Spleen/confident_promoters_noDiv_PICR.bed_atac_overlap_500bp.bed
bedtools closest -d -t first -a Results/peak_groups/Spleen/no_peaks.bed -b Results/ATAC_results/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort | awk '{if ($NF < 500) pr

In [42]:
ls Results/peak_groups/Brain/

all_peaks.bed
all_peaks.bed_atac_overlap_500bp.bed
confident_enhancers.bed
confident_enhancers.bed_atac_overlap_500bp.bed
confident_promoters.bed
confident_promoters.bed_atac_overlap_500bp.bed
confident_promoters_noDiv.bed
confident_promoters_noDiv.bed_atac_overlap_500bp.bed
confident_promoters_noDiv_PICR.bed
confident_promoters_noDiv_PICR.bed_atac_overlap_500bp.bed
no_peaks.bed
no_peaks.bed_atac_overlap_500bp.bed


In [43]:
# Intersect ATAC and TSS
dist_thresh = 100 #500 basepairs left or right
outfolder = 'Results/peak_groups/'
f_names = [
    'all_peaks','confident_promoters', 'confident_enhancers', 'confident_promoters_noDiv',
    'confident_promoters_noDiv_PICR', 'no_peaks'
]


for t in tissues_with_atac:
    print(t)
    ## Outdirectory to save in
    out_dir = os.path.join(outfolder, t)
    
    #ATAC peak file
    atac_out = glob.glob(os.path.join("Results/ATAC_results", t , '*sort'))[0]
    
    ## The different TSS files
    for f in f_names:
        f_in = os.path.join(out_dir, f) + '.bed'
        out_f = '%s_atac_overlap_%dbp.bed' % (f_in,dist_thresh)
        peaks_closest(f_in,atac_out,out_f,dist_thresh=dist_thresh)

CHO
bedtools closest -d -t first -a Results/peak_groups/CHO/all_peaks.bed -b Results/ATAC_results/CHO/ATAC_ppr.naive_overlap.narrowPeak.sort | awk '{if ($NF < 100) print }' > Results/peak_groups/CHO/all_peaks.bed_atac_overlap_100bp.bed
bedtools closest -d -t first -a Results/peak_groups/CHO/confident_promoters.bed -b Results/ATAC_results/CHO/ATAC_ppr.naive_overlap.narrowPeak.sort | awk '{if ($NF < 100) print }' > Results/peak_groups/CHO/confident_promoters.bed_atac_overlap_100bp.bed
bedtools closest -d -t first -a Results/peak_groups/CHO/confident_enhancers.bed -b Results/ATAC_results/CHO/ATAC_ppr.naive_overlap.narrowPeak.sort | awk '{if ($NF < 100) print }' > Results/peak_groups/CHO/confident_enhancers.bed_atac_overlap_100bp.bed
bedtools closest -d -t first -a Results/peak_groups/CHO/confident_promoters_noDiv.bed -b Results/ATAC_results/CHO/ATAC_ppr.naive_overlap.narrowPeak.sort | awk '{if ($NF < 100) print }' > Results/peak_groups/CHO/confident_promoters_noDiv.bed_atac_overlap_100bp.

bedtools closest -d -t first -a Results/peak_groups/Spleen/confident_enhancers.bed -b Results/ATAC_results/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort | awk '{if ($NF < 100) print }' > Results/peak_groups/Spleen/confident_enhancers.bed_atac_overlap_100bp.bed
bedtools closest -d -t first -a Results/peak_groups/Spleen/confident_promoters_noDiv.bed -b Results/ATAC_results/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort | awk '{if ($NF < 100) print }' > Results/peak_groups/Spleen/confident_promoters_noDiv.bed_atac_overlap_100bp.bed
bedtools closest -d -t first -a Results/peak_groups/Spleen/confident_promoters_noDiv_PICR.bed -b Results/ATAC_results/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort | awk '{if ($NF < 100) print }' > Results/peak_groups/Spleen/confident_promoters_noDiv_PICR.bed_atac_overlap_100bp.bed
bedtools closest -d -t first -a Results/peak_groups/Spleen/no_peaks.bed -b Results/ATAC_results/Spleen/ATAC_rep1-pr.naive_overlap.narrowPeak.sort | awk '{if ($NF < 100) pr

### Use wc -l to determine the number of peaks that are near ATAC, compared to total


In [46]:
from subprocess import check_output

def wc(filename):
    return int(check_output(["wc", "-l", filename]).split()[0])


In [50]:
dist_thresholds = [100,500]
atac_tss_df = pd.DataFrame()
for t in tissues_with_atac:
    print(t)
    out_dir = os.path.join(outfolder, t)
    ## The different TSS files
    for f in f_names:
        f_in = os.path.join(out_dir, f) + '.bed'
        
        for d in dist_thresholds:
            out_f = '%s_atac_overlap_%dbp.bed' % (f_in,d)
            print(f_in)
            pre_filt = wc(f_in)
            print(out_f)
            post_filt = wc(out_f)
            name = os.path.basename(f_in)
            frac = 1.0*post_filt/pre_filt
            atac_tss_df.at[t,name] = pre_filt
            name_atac = '_with_ATAC_%d' % d
            ## Add to dataframe
            
            atac_tss_df.at[t,name + name_atac] = post_filt
            atac_tss_df.at[t,name + name_atac + '_fraction'] = frac


CHO
Results/peak_groups/CHO/all_peaks.bed
Results/peak_groups/CHO/all_peaks.bed_atac_overlap_100bp.bed
Results/peak_groups/CHO/all_peaks.bed
Results/peak_groups/CHO/all_peaks.bed_atac_overlap_500bp.bed
Results/peak_groups/CHO/confident_promoters.bed
Results/peak_groups/CHO/confident_promoters.bed_atac_overlap_100bp.bed
Results/peak_groups/CHO/confident_promoters.bed
Results/peak_groups/CHO/confident_promoters.bed_atac_overlap_500bp.bed
Results/peak_groups/CHO/confident_enhancers.bed
Results/peak_groups/CHO/confident_enhancers.bed_atac_overlap_100bp.bed
Results/peak_groups/CHO/confident_enhancers.bed
Results/peak_groups/CHO/confident_enhancers.bed_atac_overlap_500bp.bed
Results/peak_groups/CHO/confident_promoters_noDiv.bed
Results/peak_groups/CHO/confident_promoters_noDiv.bed_atac_overlap_100bp.bed
Results/peak_groups/CHO/confident_promoters_noDiv.bed
Results/peak_groups/CHO/confident_promoters_noDiv.bed_atac_overlap_500bp.bed
Results/peak_groups/CHO/confident_promoters_noDiv_PICR.bed
R

Results/peak_groups/Spleen/no_peaks.bed
Results/peak_groups/Spleen/no_peaks.bed_atac_overlap_100bp.bed
Results/peak_groups/Spleen/no_peaks.bed
Results/peak_groups/Spleen/no_peaks.bed_atac_overlap_500bp.bed
BMDMwt
Results/peak_groups/BMDMwt/all_peaks.bed
Results/peak_groups/BMDMwt/all_peaks.bed_atac_overlap_100bp.bed
Results/peak_groups/BMDMwt/all_peaks.bed
Results/peak_groups/BMDMwt/all_peaks.bed_atac_overlap_500bp.bed
Results/peak_groups/BMDMwt/confident_promoters.bed
Results/peak_groups/BMDMwt/confident_promoters.bed_atac_overlap_100bp.bed
Results/peak_groups/BMDMwt/confident_promoters.bed
Results/peak_groups/BMDMwt/confident_promoters.bed_atac_overlap_500bp.bed
Results/peak_groups/BMDMwt/confident_enhancers.bed
Results/peak_groups/BMDMwt/confident_enhancers.bed_atac_overlap_100bp.bed
Results/peak_groups/BMDMwt/confident_enhancers.bed
Results/peak_groups/BMDMwt/confident_enhancers.bed_atac_overlap_500bp.bed
Results/peak_groups/BMDMwt/confident_promoters_noDiv.bed
Results/peak_groups/

In [51]:
atac_tss_df

,all_peaks.bed,all_peaks.bed_with_ATAC_100,all_peaks.bed_with_ATAC_100_fraction,all_peaks.bed_with_ATAC_500,all_peaks.bed_with_ATAC_500_fraction,confident_promoters.bed,confident_promoters.bed_with_ATAC_100,confident_promoters.bed_with_ATAC_100_fraction,confident_promoters.bed_with_ATAC_500,confident_promoters.bed_with_ATAC_500_fraction,...,confident_promoters_noDiv_PICR.bed,confident_promoters_noDiv_PICR.bed_with_ATAC_100,confident_promoters_noDiv_PICR.bed_with_ATAC_100_fraction,confident_promoters_noDiv_PICR.bed_with_ATAC_500,confident_promoters_noDiv_PICR.bed_with_ATAC_500_fraction,no_peaks.bed,no_peaks.bed_with_ATAC_100,no_peaks.bed_with_ATAC_100_fraction,no_peaks.bed_with_ATAC_500,no_peaks.bed_with_ATAC_500_fraction
CHO,54931.0,46443.0,0.845479,49900.0,0.908412,5271.0,4878.0,0.925441,5124.0,0.972112,...,11117.0,9096.0,0.818206,10300.0,0.926509,23724.0,19227.0,0.810445,19531.0,0.823259
Brain,37316.0,29313.0,0.785534,31252.0,0.837496,4080.0,3916.0,0.959804,4013.0,0.983578,...,13374.0,10650.0,0.796321,11876.0,0.887992,23792.0,20947.0,0.880422,21249.0,0.893115
Liver,56905.0,8847.0,0.155470,10891.0,0.191389,3426.0,1171.0,0.341798,1371.0,0.400175,...,11272.0,3212.0,0.284954,3993.0,0.354241,24409.0,10256.0,0.420173,10732.0,0.439674
Lung,45148.0,25302.0,0.560423,28473.0,0.630659,4036.0,3512.0,0.870168,3738.0,0.926165,...,12684.0,8922.0,0.703406,10120.0,0.797856,24324.0,18765.0,0.771460,19223.0,0.790289
Kidney,50389.0,35906.0,0.712576,38668.0,0.767390,3813.0,3604.0,0.945188,3721.0,0.975872,...,12050.0,9710.0,0.805809,10801.0,0.896349,24322.0,21619.0,0.888866,21906.0,0.900666
Spleen,38631.0,21151.0,0.547514,23424.0,0.606352,3117.0,2655.0,0.851781,2792.0,0.895733,...,11313.0,7732.0,0.683462,8728.0,0.771502,24586.0,18175.0,0.739242,18624.0,0.757504
BMDMwt,85558.0,38630.0,0.451507,44369.0,0.518584,3138.0,2998.0,0.955386,3108.0,0.990440,...,10751.0,8516.0,0.792112,9517.0,0.885220,24294.0,21493.0,0.884704,21747.0,0.895159


In [54]:
col_names = np.array(atac_tss_df.columns.values)
#to_drop = col_names[map(lambda x: 'fraction' not in x or '100' not in x,atac_tss_df.columns.values)]
#output = atac_tss_df.drop(to_drop,axis=1) 
output = atac_tss_df.copy()
col_names = np.array(output.columns.values)
for i,name in enumerate((col_names)):
    name = name.replace('.bed','')
    #name = name.replace('fraction','')
    name = name.replace('ATAC','ATAC peak within')
    #name = 'Fraction of ' + name
    #name = name + 'bp'
    col_names[i] = name.replace('_',' ')
output.columns = col_names



,all peaks,all peaks with ATAC peak within 100,all peaks with ATAC peak within 100 fraction,all peaks with ATAC peak within 500,all peaks with ATAC peak within 500 fraction,confident promoters,confident promoters with ATAC peak within 100,confident promoters with ATAC peak within 100 fraction,confident promoters with ATAC peak within 500,confident promoters with ATAC peak within 500 fraction,...,confident promoters noDiv PICR,confident promoters noDiv PICR with ATAC peak within 100,confident promoters noDiv PICR with ATAC peak within 100 fraction,confident promoters noDiv PICR with ATAC peak within 500,confident promoters noDiv PICR with ATAC peak within 500 fraction,no peaks,no peaks with ATAC peak within 100,no peaks with ATAC peak within 100 fraction,no peaks with ATAC peak within 500,no peaks with ATAC peak within 500 fraction
CHO,54931.0,46443.0,0.845479,49900.0,0.908412,5271.0,4878.0,0.925441,5124.0,0.972112,...,11117.0,9096.0,0.818206,10300.0,0.926509,23724.0,19227.0,0.810445,19531.0,0.823259
Brain,37316.0,29313.0,0.785534,31252.0,0.837496,4080.0,3916.0,0.959804,4013.0,0.983578,...,13374.0,10650.0,0.796321,11876.0,0.887992,23792.0,20947.0,0.880422,21249.0,0.893115
Liver,56905.0,8847.0,0.155470,10891.0,0.191389,3426.0,1171.0,0.341798,1371.0,0.400175,...,11272.0,3212.0,0.284954,3993.0,0.354241,24409.0,10256.0,0.420173,10732.0,0.439674
Lung,45148.0,25302.0,0.560423,28473.0,0.630659,4036.0,3512.0,0.870168,3738.0,0.926165,...,12684.0,8922.0,0.703406,10120.0,0.797856,24324.0,18765.0,0.771460,19223.0,0.790289
Kidney,50389.0,35906.0,0.712576,38668.0,0.767390,3813.0,3604.0,0.945188,3721.0,0.975872,...,12050.0,9710.0,0.805809,10801.0,0.896349,24322.0,21619.0,0.888866,21906.0,0.900666
Spleen,38631.0,21151.0,0.547514,23424.0,0.606352,3117.0,2655.0,0.851781,2792.0,0.895733,...,11313.0,7732.0,0.683462,8728.0,0.771502,24586.0,18175.0,0.739242,18624.0,0.757504
BMDMwt,85558.0,38630.0,0.451507,44369.0,0.518584,3138.0,2998.0,0.955386,3108.0,0.990440,...,10751.0,8516.0,0.792112,9517.0,0.885220,24294.0,21493.0,0.884704,21747.0,0.895159


In [55]:
output.transpose()

,CHO,Brain,Liver,Lung,Kidney,Spleen,BMDMwt
all peaks,54931.000000,37316.000000,56905.000000,45148.000000,50389.000000,38631.000000,85558.000000
all peaks with ATAC peak within 100,46443.000000,29313.000000,8847.000000,25302.000000,35906.000000,21151.000000,38630.000000
all peaks with ATAC peak within 100 fraction,0.845479,0.785534,0.155470,0.560423,0.712576,0.547514,0.451507
all peaks with ATAC peak within 500,49900.000000,31252.000000,10891.000000,28473.000000,38668.000000,23424.000000,44369.000000
all peaks with ATAC peak within 500 fraction,0.908412,0.837496,0.191389,0.630659,0.767390,0.606352,0.518584
confident promoters,5271.000000,4080.000000,3426.000000,4036.000000,3813.000000,3117.000000,3138.000000
confident promoters with ATAC peak within 100,4878.000000,3916.000000,1171.000000,3512.000000,3604.000000,2655.000000,2998.000000
confident promoters with ATAC peak within 100 fraction,0.925441,0.959804,0.341798,0.870168,0.945188,0.851781,0.955386
confident promoters with ATAC peak within 500,5124.000000,4013.000000,1371.000000,3738.000000,3721.000000,2792.000000,3108.000000
confident promoters with ATAC peak within 500 fraction,0.972112,0.983578,0.400175,0.926165,0.975872,0.895733,0.990440


---------

---------